In [1]:
from sagemaker.pytorch import PyTorch

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# configuration for running training on smdistributed Data Parallel
# distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

In [3]:
import sagemaker

In [4]:
session = sagemaker.LocalSession()
session.config = {'local': {'local_code': True}}
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name lioptim_udesa to get Role path.


In [5]:
sagemaker.image_uris.retrieve('pytorch', region='us-east-2', image_scope='training',
                             py_version='py310', version='2.0', instance_type='ml.t3.large')

'763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-training:2.0-cpu-py310'

In [6]:
pytorch_estimator = PyTorch(entry_point='train.py',
                            source_dir='code',
                            role=role,
                            base_job_name='NLISTS',
                            instance_type= 'ml.g4dn.xlarge',
                            py_version="py310",
                            instance_count=1,
                            framework_version='2.0',
                            output_path='s3://nli-sts/logs',
                            script_mode=True,
                            hyperparameters = {
                                'model-name': 'dccuchile/bert-base-spanish-wwm-cased',
                                'epoch': 1,
                                'step':0,
                                'lr':2e-5
                            },
                            # distribution=distribution
                       )

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [9]:
pytorch_estimator.fit({'train':'s3://nli-sts/eval-data/', 'validation':'s3://nli-sts/eval-data/'})

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: NLISTS-2023-10-25-18-21-12-135


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: The account-level service limit 'ml.g4dn.xlarge for training job usage' is 1 Instances, with current utilization of 1 Instances and a request delta of 1 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.